In [36]:
import pandas as pd

from scipy.stats import wilcoxon

In [41]:
dtc = pd.read_csv('effect_presence_dtc.csv')
dtr = pd.read_csv('effect_presence_dtr.csv')

In [38]:
def do_testing(pdf, metric):
    params = pdf['params'].drop_duplicates().values
    pdf0 = pdf[pdf['params'] == params[0]].sort_values('fold')
    pdf1 = pdf[pdf['params'] == params[1]].sort_values('fold')

    return pd.Series({
        'wilcoxon': wilcoxon(pdf0[metric], pdf1[metric], zero_method='zsplit').pvalue
    })

In [44]:
def summarize(data, metric):
    pd_wilcoxon = data.groupby('name')\
                    .apply(lambda x: do_testing(x, metric))\
                    .reset_index(drop=False)

    grouped = data.groupby(['name', 'params'])\
                    .agg({metric: 'mean'})\
                    .reset_index(drop=False)

    grouped['label'] = grouped.apply(lambda x: 'less' if eval(x['params'])['operator'] == '<' else 'leq', axis=1)

    pivoted = grouped.pivot(index='name', columns='label', values=metric)\
                .reset_index(drop=False)\
                .rename(columns={'leq': f'{metric}_leq', 'less': f'{metric}_less'})

    return pd.merge(pivoted, pd_wilcoxon, on=['name'])

In [45]:
summarize(dtc, 'auc')

,name,auc_leq,auc_less,wilcoxon
0,abalone9_18,0.650018,0.650018,1.000000
1,appendicitis,0.712056,0.711997,0.928942
2,bupa,0.626159,0.625188,0.517240
3,cleveland-0_vs_4,0.684516,0.688713,0.778081
4,ecoli1,0.832220,0.832497,0.725537
5,glass0,0.783446,0.783446,1.000000
6,haberman,0.561040,0.561384,0.918423
7,hepatitis,0.650958,0.653467,0.024033
8,lymphography-normal-fibrosis,0.899964,0.905038,0.596259
9,mammographic,0.786932,0.788242,0.021111


In [46]:
summarize(dtr, 'r2')

,name,r2_leq,r2_less,wilcoxon
0,airfoil,0.860597,0.860579,1.759729e-07
1,autoMPG6,0.763590,0.765539,1.265005e-09
2,baseball,0.405100,0.403743,4.150450e-01
3,cpu_performance,0.789304,0.785683,7.118748e-03
4,daily-demand,0.660710,0.661539,3.639243e-01
5,diabetes,-0.878408,-0.881574,7.912182e-01
6,excitation_current,0.999818,0.999818,3.497921e-02
7,laser,0.922193,0.924267,4.294329e-04
8,maternal_health_risk,0.711601,0.710200,1.124098e-03
9,medical_cost,0.530427,0.530943,1.474142e-06
